# 0. Importing Necessary Libraries

# 1. Transaction Data Simulator

For confidentiality reasons, it is hard to find data related to fraud detection problems, despite its importance. Therefore, I will use a transaction data simulator to generate data for this project. For more details about the simulator, please refer to a [recent book about ML for fraud detection](https://fraud-detection-handbook.github.io/fraud-detection-handbook/Foreword.html). In the following, I will briefly explain the main ideas behind the transactions data generating process.

The following table describes the features that will be present in each transaction.

| Transaction Feature | Description |
|:-------------------:|:-----------:|
|TRANSACTION_ID | Unique identifier for the transaction.|
|TX_DATETIME | Date and time of the transaction. |
|CUSTOMER_ID | Unique identifier of the customer who made the transaction. |
|TERMINAL_ID | Unique identifier of the terminal from which the transaction was made. |
|TX_AMOUNT | The amount of the transaction. I will assume it is in US dollars. |
|TX_FRAUD | Indicator variable for frauds (0 - Legitimate; 1 - Fraud).|

